In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
import os

In [ ]:
# Loading the env variables

load_dotenv()

In [ ]:
# State variable

class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
# Defining our LLM (llama-3.1-8b-instant) from Groq for faster inference

llm = ChatGroq(model="llama-3.1-8b-instant")

In [ ]:
# Chat node in our graph, that communicates to the LLM

def chat_node (state: ChatState) -> ChatState:
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

In [ ]:
graph = StateGraph(ChatState)

# in RAM memory for out chatbot
checkpointer = MemorySaver()

# Nodes in our graph
graph.add_node("chat_node", chat_node)

# Edges connecting the nodes
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

In [ ]:
chatbot = graph.compile(checkpointer=checkpointer)
chatbot

In [18]:
# Client Side

thread_id = "1"

while True:
    user_message = input("Type Here: ")
    print("User: ",user_message)
    
    if user_message.strip().lower() in ["end", "exit", "bye", "sakyo"]:
        break
    
    # Config variable that manages the persistence
    
    config = {"configurable" : {"thread_id" : thread_id}}
    
    response = chatbot.invoke({"messages" : [HumanMessage(content=user_message)]}, config=config)
    
    print("AI: ",response["messages"][-1].content)
    print()

User:  hey
AI:  Hey Rahul. Feeling a bit chatty today? Want to keep talking about fitness or move on to something else?

User:  i am sad
AI:  I'm so sorry to hear that, Rahul. It's okay to feel sad sometimes, and I'm here to listen if you want to talk about what's on your mind.

If you're comfortable, can you tell me a bit more about what's making you feel sad? Sometimes sharing your feelings with someone can help you feel a little better.

If you're not ready to talk about it, that's okay too. I can suggest some things that might help you feel a bit better, like taking a deep breath, going for a walk, or doing something you enjoy.

You're not alone, Rahul. I'm here to support you.

User:  
AI:  It can be hard to put into words what's going on. Take your time, and when you're ready, feel free to talk about it or just listen to some calming words.

Remember, you're strong and capable, and things might seem dark right now, but they can get better. Sometimes, all we need is someone to lis